# Preparing the Data for Machine Learning
This notebook is used to prepare the data for machine learning by using Scikit Learn Pipelines to perform data imputation.

In [24]:
# For Deepnote to be able to use the custom libraries in the parent ../lib folder
import sys
sys.path.append('..')

In [25]:
import pandas as pd
from lib.read_data import read_and_join_output_file
from lib.impute import create_transformation_pipelines
from lib.viz import draw_missing_data_chart, draw_corr_heatmap, draw_components_variance_chart, biplot, draw_feature_importance
from sklearn.decomposition import PCA
from sklearn import set_config

First we load and join all the datasets resulting from the ETL process and initialize some variables.

We have two potential targets for supervised and unsupervised learning, which we remove from the list of features:
* `GSE_GWE` - The Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface
* `SHORTAGE_COUNT` -  The number of reported well shortages

In [26]:

indv_feature_dict, all_features_df = read_and_join_output_file()
feature_columns = list(all_features_df.columns)
RANDOM_SEED = 42
feature_columns.remove("GSE_GWE")
feature_columns.remove("SHORTAGE_COUNT")
X = all_features_df[feature_columns]
y = all_features_df["GSE_GWE"]

## Missing Data

In [27]:
X.sample(5)

,,CROP_C,CROP_C6,CROP_D10,CROP_D12,CROP_D13,CROP_D14,CROP_D15,CROP_D16,CROP_D3,CROP_D5,...,GROUNDSURFACEELEVATION_AVG,STATICWATERLEVEL_AVG,TOPOFPERFORATEDINTERVAL_AVG,TOTALDRILLDEPTH_AVG,TOTALCOMPLETEDDEPTH_AVG,WELLYIELD_AVG,WELL_COUNT_AGRICULTURE,WELL_COUNT_DOMESTIC,WELL_COUNT_INDUSTRIAL,WELL_COUNT_PUBLIC
TOWNSHIP_RANGE,YEAR,,,,,,,,,,,,,,,,,,,,,
T06S R12E,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.872000,81.000000,222.400000,388.000000,379.400000,815.000000,0.0,4.0,0.0,1.0
T16S R22E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.674595,238.424242,355.448276,460.675676,455.675676,109.074074,8.0,29.0,0.0,0.0
T20S R18E,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
T19S R21E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72.190000,179.250000,221.571429,430.090909,412.090909,NaN,7.0,4.0,0.0,0.0
T20S R17E,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


Let's look at the features with missing data.

In [28]:
draw_missing_data_chart(X)

alt.LayerChart(...)

### Data Missing for Specific Years
Data were collected from the years 2014 to 2021 but some datasets only have data for specific years, when surveys were done/published. For example:
* Soils survey only has data for 2016
* Vegetations only has data for 2019
* Crops only has data for the the years 2014, 2016 and 2018
* Population density is available only for the years 2014 - 2020
* The reservoir water `PCT_OF_CAPACITY` is available only for the years 2018 - 2020

In [29]:
crops_columns = [col for col in X if col.startswith('CROP_')]
crops_df = X[crops_columns].dropna()
soils_columns = [col for col in X if col.startswith('SOIL_')]
soils_df = X[soils_columns].dropna()
print(f"Years present in the Soils dataset {list(crops_df.index.unique(level='YEAR'))}")
print(f"Years present in the Crops dataset {list(soils_df.index.unique(level='YEAR'))}")

Years present in the Soils dataset ['2014', '2016', '2018']
Years present in the Crops dataset ['2016']


### Data Missing for Specific Township-Ranges
The Well Completion Reports dataset has data for all the years but have missing data for some specific Township-Ranges. Typically, if no wells were drilled in a specific Township-Range during the 2014-2020 period, then there is no data for that Township-Range for any of the following features:
* `TOTALDRILLDEPTH_AVG`
* `WELLYIELD_AVG`
* `STATICWATERLEVEL_AVG`
* `TOPOFPERFORATEDINTERVAL_AVG`
* `BOTTOMOFPERFORATEDINTERVAL_AVG`
* `GROUNDSURFACEELEVATION_AVG`
* `TOTALCOMPLETEDDEPTH_AVG`

Wells can also be reported with incomplete data, which means that some of the above features data could be missing for some Township-Ranges, even if wells were reported in those Township-Range.

In [30]:
all_township_ranges = set(X.index.unique(level="TOWNSHIP_RANGE"))
wells_columns = [col for col in X if col.endswith('_AVG') or col == "TOWNSHIP_RANGE"]
wells_df = X[wells_columns].dropna()
missing_township_ranges = all_township_ranges - set(wells_df.index.unique(level="TOWNSHIP_RANGE"))
print(f"There are {len(missing_township_ranges)} out of {len(all_township_ranges)} Township-Ranges with missing well completion report data: {missing_township_ranges}")

There are 169 out of 478 Township-Ranges with missing well completion report data: {'T16S R13E', 'T30S R23E', 'T12N R22W', 'T30S R31E', 'T19S R19E', 'T28S R21E', 'T03S R14E', 'T12S R11E', 'T28S R24E', 'T03N R04E', 'T20S R21E', 'T03N R05E', 'T26S R20E', 'T29S R21E', 'T27S R22E', 'T30S R20E', 'T32S R25E', 'T15S R10E', 'T24S R20E', 'T08S R12E', 'T13S R14E', 'T32S R23E', 'T11S R15E', 'T31S R22E', 'T12S R13E', 'T25S R26E', 'T10N R23W', 'T18S R17E', 'T22S R16E', 'T10S R19E', 'T07S R07E', 'T18S R14E', 'T13S R16E', 'T26S R22E', 'T12N R24W', 'T27S R19E', 'T08S R11E', 'T12S R16E', 'T25S R20E', 'T10S R12E', 'T20S R14E', 'T07N R08E', 'T11N R19W', 'T19S R15E', 'T07S R09E', 'T22S R21E', 'T14S R25E', 'T10N R22W', 'T02S R03E', 'T29S R20E', 'T10S R21E', 'T06S R16E', 'T22S R29E', 'T24S R22E', 'T06S R07E', 'T08S R10E', 'T22S R23E', 'T20S R28E', 'T02S R13E', 'T10S R08E', 'T10S R20E', 'T09S R11E', 'T17S R16E', 'T13S R13E', 'T07S R16E', 'T13S R10E', 'T23S R21E', 'T02N R05E', 'T15S R11E', 'T15S R13E', 'T17S 

## Data Imputation
### Imputation Strategies
To impute the missing data we will use the following strategies
1. We assume little year-to-year variation in Crops, Soils and Vegetation. The missing Crops data will thus be imputed from the previous year (e.g. the 2015 data will be set as the 2014 data). For the Soils and Vegetation where we only have data for 1 year, the missing data will all be imputed from the available year.
2. The 2021 population density data will be estimated based on the 2020 population density and the 2019-2020 trend.
3. For the pre-2018 missing reservoir water `PCT_OF_CAPACITY` data, as California was affected by sever droughts during those years, we will impute missing data by taking the **minimum** `PCT_OF_CAPACITY` for that Township-Range in the post 2018 data.
4. For the well completion reports' features with missing we will use 2 distinct strategies:
    * For the  `GROUNDSURFACEELEVATION_AVG` feature we will use the median values over all the years for that Township-Range. For Township-Ranges with no data at all for any of the 2014-2020 years, we will use the median value over all Township-Ranges.
    * For the other features they will be set to 0, since these are well measurements and missing data are mainly due to no wells being drilled in that Township-Range and year.

In [31]:
impute_pipeline, columns = create_transformation_pipelines(X)

In [32]:
X_impute = impute_pipeline.fit_transform(X)
X_impute_df = pd.DataFrame(X_impute, index=X.index, columns=columns)
X_impute_df.sample()
draw_missing_data_chart(X_impute_df)

alt.LayerChart(...)

In [33]:
set_config(display="diagram")
display(impute_pipeline)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=MinMaxScaler(),
                                   transformers=[('wcr',
                                                  Pipeline(steps=[('imputer',
                                                                   PandasSimpleImputer(fill_value=0,
                                                                                       strategy='constant')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['TOTALDRILLDEPTH_AVG',
                                                   'WELLYIELD_AVG',
                                                   'STATICWATERLEVEL_AVG',
                                                   'TOPOFPERFORATEDINTERVAL_AVG',
                                                   'BOTTOMOFPERFORATEDINTERVAL_AVG',
                                                   'TOTALCOMPLETE...
                                                                   GroupImputer(aggregation_func='min',
                                                                                group_by_cols=['TOWNSHIP_RANGE'],
                                                                                impute_for_col='PCT_OF_CAPACITY')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['PCT_OF_CAPACITY']),
                                                 ('gse',
                                                  Pipeline(steps=[('imputer',
                                                                   GroupImputer(aggregation_func='median',
                                                                                group_by_cols=['TOWNSHIP_RANGE'],
                                                                                impute_for_col='GROUNDSURFACEELEVATION_AVG')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['GROUNDSURFACEELEVATION_AVG'])]))])

In [34]:
X_impute_df["CROP_C6"]

TOWNSHIP_RANGE  YEAR
T01N R02E       2014    0.005554
                2015    0.005554
                2016    0.004971
                2017    0.004971
                2018    0.005110
                          ...   
T32S R30E       2017    0.000000
                2018    0.000000
                2019    0.000000
                2020    0.000000
                2021    0.000000
Name: CROP_C6, Length: 3824, dtype: float64

In [35]:
X["CROP_C6"]

TOWNSHIP_RANGE  YEAR
T01N R02E       2014    0.005554
                2015         NaN
                2016    0.004971
                2017         NaN
                2018    0.005110
                          ...   
T32S R30E       2017         NaN
                2018    0.000000
                2019         NaN
                2020         NaN
                2021         NaN
Name: CROP_C6, Length: 3824, dtype: float64

In [36]:
X_impute_df

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014             0.000000       0.000489              0.020868   
               2015             0.000000       0.000000              0.000000   
               2016             0.000000       0.003259              0.036728   
               2017             0.066667       0.006410              0.025876   
               2018             0.053651       0.000652              0.063022   
...                                  ...            ...                   ...   
T32S R30E      2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   
               2021             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R02E      2014                     0.052288   
               2015                     0.000000   
               2016                     0.084967   
               2017                     0.082789   
               2018                     0.077124   
...                                          ...   
T32S R30E      2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   
               2021                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R02E      2014                        0.075670                 0.127841   
               2015                        0.000000                 0.000000   
               2016                        0.057471                 0.056818   
               2017                        0.063857                 0.074495   
               2018                        0.052874                 0.064015   
...                                             ...                      ...   
T32S R30E      2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   
               2021                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R02E      2014                       0.010798   
               2015                       0.010798   
               2016                       0.010798   
               2017                       0.010798   
               2018                       0.010798   
...                                            ...   
T32S R30E      2017                       0.033178   
               2018                       0.033178   
               2019                       0.033178   
               2020                       0.033178   
               2021                       0.033178   

                     VEGETATION_CALIFORNIA_COAST_LIVE_OAK  \
TOWNSHIP_RANGE YEAR                                         
T01N R02E      2014                              0.002749   
               2015                              0.002749   
               2016                              0.002749   
               2017                              0.002749   
               2018                              0.002749   
...                                                   ...   
T32S R30E      2017                              0.000000 

In [37]:
impute_for_col = "PCT_OF_CAPACITY"
group_by_cols = ["TOWNSHIP_RANGE"]
impute_group_map = X.groupby(group_by_cols)[[impute_for_col]].agg("median")
return_df = X.copy()
for index, row in impute_group_map.iterrows():
    return_df.loc[index, impute_for_col].fillna(row.values[0], inplace=True)
return_df

CROP_C   CROP_C6  CROP_D10  CROP_D12  CROP_D13  CROP_D14  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014     0.0  0.005554  0.000176  0.000000  0.008879  0.003796   
               2015     NaN       NaN       NaN       NaN       NaN       NaN   
               2016     0.0  0.004971  0.001729  0.000000  0.005217  0.003099   
               2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.005110  0.001337  0.000000  0.002000  0.003084   
...                     ...       ...       ...       ...       ...       ...   
T32S R30E      2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.000000  0.000000  0.005819  0.000000  0.009827   
               2019     NaN       NaN       NaN       NaN       NaN       NaN   
               2020     NaN       NaN       NaN       NaN       NaN       NaN   
               2021     NaN       NaN       NaN       NaN       NaN       NaN   

                     CROP_D15  CROP_D16   CROP_D3   CROP_D5  ...  \
TOWNSHIP_RANGE YEAR                                          ...   
T01N R02E      2014  0.000015  0.002058  0.018790  0.010297  ...   
               2015       NaN       NaN       NaN       NaN  ...   
               2016  0.000704  0.000792  0.014696  0.008706  ...   
               2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000690  0.000612  0.013697  0.005494  ...   
...                       ...       ...       ...       ...  ...   
T32S R30E      2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000000  0.000000  0.000000  0.000000  ...   
               2019       NaN       NaN       NaN       NaN  ...   
               2020       NaN       NaN       NaN       NaN  ...   
               2021       NaN       NaN       NaN       NaN  ...   

                     GROUNDSURFACEELEVATION_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R02E      2014                   56.905000                  25.0   
               2015                         NaN                   NaN   
               2016                   20.280000                  44.0   
               2017                   41.086667                  31.0   
               2018                   69.050000                  75.5   
...                                         ...                   ...   
T32S R30E      2017                         NaN                   NaN   
               2018                         NaN                   NaN   
               2019                         NaN                   NaN   
               2020                         NaN                   NaN   
               2021                         NaN                   NaN   

                     TOPOFPERFORATEDINTERVAL_AVG  TOTALDRILLDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R02E      2014                    80.000000                  NaN   
               2015                          NaN                  NaN   
               2016                   130.000000                  NaN   
               2017                   126.666667                210.0   
               2018                   118.000000                169.0   
...                                          ...                  ...   
T32S R30E      2017                          NaN                  NaN   
               2018                          NaN                  NaN   
               2019                          NaN                  NaN   
               2020                          NaN                  NaN   
               2021                          NaN                  NaN   

                     TOTALCOMPLETEDDEPTH_AVG  WELLYIELD_AVG  \
TOWNSHIP_RANGE YEAR                                           
T01N R02E      2014               337.500000       3.000